# Portfolio Risk Analysis

This notebook demonstrates portfolio-level risk management for options.

We'll cover:
- Building a multi-position options portfolio
- Computing aggregate portfolio Greeks
- Scenario analysis with spot and volatility shocks
- Understanding delta and vega hedging

## Setup

In [ ]:
import sys
sys.path.insert(0, '..')

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from src.core.option_types import Option, OptionType, ExerciseStyle
from src.core.portfolio import (
    Position,
    Portfolio,
    portfolio_price,
    portfolio_greeks,
    scenario_pnl,
)

# Plot styling
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

## Build a Sample Portfolio

We'll create a portfolio with a mix of calls and puts at different strikes, with both long and short positions.

This represents a typical trading book or hedge portfolio.

In [ ]:
# Common parameters
spot = 100.0
rate = 0.05
vol = 0.20
T = 0.5  # 6 months

# Create individual options
call_90 = Option(spot, 90, rate, vol, T, OptionType.CALL, ExerciseStyle.EUROPEAN)
call_100 = Option(spot, 100, rate, vol, T, OptionType.CALL, ExerciseStyle.EUROPEAN)
call_110 = Option(spot, 110, rate, vol, T, OptionType.CALL, ExerciseStyle.EUROPEAN)
put_90 = Option(spot, 90, rate, vol, T, OptionType.PUT, ExerciseStyle.EUROPEAN)
put_100 = Option(spot, 100, rate, vol, T, OptionType.PUT, ExerciseStyle.EUROPEAN)

# Build portfolio
# This is roughly a "risk reversal" plus some extra positions
positions = [
    Position(call_100, 10),    # Long 10 ATM calls
    Position(put_100, -5),     # Short 5 ATM puts
    Position(call_110, -3),    # Short 3 OTM calls (covered call)
    Position(put_90, 2),       # Long 2 OTM puts (protection)
    Position(call_90, 1),      # Long 1 ITM call
]

portfolio = Portfolio(positions)

print("Portfolio Composition:")
print("=" * 55)
print(f"{'Type':<8} {'Strike':<8} {'Qty':<8} {'Direction'}")
print("-" * 55)
for pos in positions:
    opt_type = pos.option.option_type.value.upper()
    strike = pos.option.strike
    qty = pos.quantity
    direction = "LONG" if qty > 0 else "SHORT"
    print(f"{opt_type:<8} {strike:<8.0f} {abs(qty):<8.0f} {direction}")

## Portfolio Price and Greeks

Let's compute the aggregate portfolio metrics.

In [ ]:
# Compute portfolio price
price = portfolio_price(portfolio)

# Compute portfolio Greeks
greeks = portfolio_greeks(portfolio)

print("Portfolio Summary")
print("=" * 40)
print(f"Total Price:  ${price:,.2f}")
print()
print("Aggregate Greeks:")
print(f"  Delta: {greeks['delta']:+.2f}")
print(f"  Gamma: {greeks['gamma']:+.4f}")
print(f"  Vega:  {greeks['vega']:+.2f}")
print(f"  Theta: {greeks['theta']:+.2f} (per year)")
print(f"  Rho:   {greeks['rho']:+.2f}")

### Interpreting the Greeks

- **Delta > 0**: Portfolio gains when spot increases (net long exposure)
- **Gamma > 0**: Delta increases as spot rises (convexity, good for long options)
- **Vega > 0**: Portfolio gains when volatility increases (net long vol)
- **Theta < 0**: Portfolio loses value over time (time decay)

## Scenario Analysis

Let's stress test the portfolio across a grid of spot and volatility shocks.

In [ ]:
# Define shock scenarios
spot_shocks = [-20, -10, 0, 10, 20]  # Absolute changes in spot
vol_shocks = [-0.05, 0.0, 0.05]      # Absolute changes in vol

# Run scenario analysis
results = scenario_pnl(portfolio, spot_shocks, vol_shocks)

# Convert to DataFrame for display
df = pd.DataFrame(results)
df['spot_shock'] = df['spot_shock'].astype(int)
df['vol_shock'] = df['vol_shock'].apply(lambda x: f"{x:+.0%}")
df['price'] = df['price'].apply(lambda x: f"${x:,.2f}")
df['pnl'] = df['pnl'].apply(lambda x: f"${x:+,.2f}")

print("Scenario P&L Results:")
print(df[['spot_shock', 'vol_shock', 'price', 'pnl']].to_string(index=False))

In [ ]:
# Create P&L heatmap
results = scenario_pnl(portfolio, spot_shocks, vol_shocks)

# Reshape for heatmap
pnl_matrix = np.zeros((len(vol_shocks), len(spot_shocks)))

for result in results:
    i = vol_shocks.index(result['vol_shock'])
    j = spot_shocks.index(result['spot_shock'])
    pnl_matrix[i, j] = result['pnl']

# Plot heatmap
fig, ax = plt.subplots(figsize=(10, 6))

im = ax.imshow(pnl_matrix, cmap='RdYlGn', aspect='auto')

# Labels
ax.set_xticks(range(len(spot_shocks)))
ax.set_yticks(range(len(vol_shocks)))
ax.set_xticklabels([f"{s:+d}" for s in spot_shocks])
ax.set_yticklabels([f"{v:+.0%}" for v in vol_shocks])
ax.set_xlabel('Spot Shock ($)', fontsize=12)
ax.set_ylabel('Vol Shock', fontsize=12)
ax.set_title('Portfolio P&L Heatmap', fontsize=14)

# Add text annotations
for i in range(len(vol_shocks)):
    for j in range(len(spot_shocks)):
        text = f"${pnl_matrix[i, j]:+.0f}"
        color = 'white' if abs(pnl_matrix[i, j]) > 20 else 'black'
        ax.text(j, i, text, ha='center', va='center', color=color, fontsize=10)

# Colorbar
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('P&L ($)', fontsize=10)

plt.tight_layout()
plt.show()

### Reading the Heatmap

- **Green cells**: Portfolio gains
- **Red cells**: Portfolio loses
- **Row pattern**: Effect of volatility at each spot level
- **Column pattern**: Effect of spot at each vol level

## Delta and Vega Hedging

### What is Hedging?

Hedging means reducing unwanted risk exposures. For an options portfolio:

- **Delta hedging**: Neutralize sensitivity to spot price changes
- **Vega hedging**: Neutralize sensitivity to volatility changes

### How to Hedge

**Delta Hedging**: Trade the underlying stock
- Current portfolio delta: **{delta:.2f}**
- To delta hedge: Short **{delta:.0f}** shares of stock
- Result: Portfolio P&L won't change (much) for small spot moves

**Vega Hedging**: Trade other options
- Current portfolio vega: **{vega:.2f}**
- To vega hedge: Take opposite vega position with other options
- Result: Portfolio won't be affected by implied vol changes

In [ ]:
# Delta hedging example
delta = greeks['delta']
vega = greeks['vega']

print("Hedging Recommendations")
print("=" * 50)
print()
print(f"Current Delta: {delta:+.2f}")
print(f"  → To delta-neutral: SHORT {abs(delta):.0f} shares")
print()
print(f"Current Vega: {vega:+.2f}")
if vega > 0:
    print(f"  → To vega-neutral: SHORT vega via options")
else:
    print(f"  → To vega-neutral: LONG vega via options")
print()
print("After delta hedging:")
print("  - Small spot moves won't affect P&L")
print("  - Still exposed to gamma (large moves)")
print("  - Still exposed to vega (vol changes)")

### Why Hedge?

1. **Market makers**: Capture bid-ask spread without directional risk
2. **Risk management**: Limit losses in adverse scenarios
3. **Isolate exposures**: Trade only the risk you want (e.g., vol)

### Limitations of Hedging

- **Discrete hedging**: Can't rebalance continuously, so residual risk remains
- **Transaction costs**: Each hedge trade costs money
- **Model risk**: Greeks are estimates based on model assumptions
- **Gamma risk**: Large moves can blow through delta hedges

## Summary

| Concept | Description | Action |
|---------|-------------|--------|
| Portfolio Greeks | Aggregate risk metrics | Sum of position Greeks |
| Delta Hedge | Neutralize spot exposure | Trade underlying stock |
| Vega Hedge | Neutralize vol exposure | Trade other options |
| Scenario Analysis | Stress test across shocks | Grid of spot × vol |

Portfolio risk management is about understanding your exposures and deciding which risks to keep and which to hedge away.